In [ ]:
#| default_exp create_umap_layout

import numpy as np
import umap

/home/willsa/git/bedmap-dev/.venv/lib/python3.12/site-packages/umap/__init__.py:36: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


# Create umap layout(s)

In [ ]:
#| export

def umap_2d(embeddings: np.ndarray, n_neighbors: int = 15, min_dist: float = 0.1,
                metric: str = "cosine", random_state: int = 42) -> np.ndarray:
    """
    Create a UMAP layout from embeddings.

    Args:
        embeddings: np.ndarray, shape (n_samples, n_features)
        n_neighbors: int, default=15
        min_dist: float, default=0.1
        n_components: int, default=2
    """
    return umap.UMAP(n_neighbors=n_neighbors, min_dist=min_dist,
                     metric=metric, random_state=random_state).fit_transform(embeddings)

In [ ]:
#| hide

umap_2d(np.random.rand(100, 10), n_neighbors=15, min_dist=0.1, metric="cosine", random_state=42)

array([[2.2622879, 3.4954352],
       [4.1649737, 6.3780503],
       [2.4475107, 5.165305 ],
       [6.204571 , 2.7574754],
       [5.7717943, 4.7820377],
       [5.2059975, 5.6740336],
       [6.0497327, 4.5488214],
       [3.697823 , 5.4977465],
       [2.527554 , 5.7558713],
       [2.5686183, 5.4819717],
       [4.9252753, 6.539931 ],
       [3.9040804, 5.4743133],
       [2.3970942, 3.52889  ],
       [4.3505287, 3.859867 ],
       [6.6205683, 5.3519745],
       [4.041339 , 2.9313416],
       [4.3225327, 5.2423997],
       [6.048555 , 5.3204417],
       [4.7763395, 3.9546995],
       [4.374961 , 4.7448277],
       [5.5991073, 3.6187093],
       [6.456001 , 2.897146 ],
       [2.1105855, 3.8307745],
       [5.061707 , 3.5033538],
       [5.94312  , 4.4439173],
       [2.954937 , 3.8101187],
       [2.751163 , 3.9146104],
       [6.622923 , 2.9809697],
       [2.3812425, 4.762208 ],
       [3.0693274, 4.384201 ],
       [5.6486564, 6.320115 ],
       [4.84412  , 6.552604 ],
       [